In [ ]:
import cv2
import glob
import matplotlib.pyplot as plt
import numpy as np
import os
import pickle
import photogrammetry as pg
import seaborn as sns
import time

# %matplotlib qt

plt.style.use('default')

%load_ext autoreload
%autoreload 2

In [ ]:
# in case a new board type is needed
# pg.generate_charuco_board(pg.BOARD_VERT_SHAPE, pg.BOARD_SQUARE_SIZE, pg.BOARD_ARUCO_SIZE, aruco_dict=cv2.aruco.DICT_4X4_1000, gen_png=True, invert=True)

In [ ]:
%autoreload 2

run = '20230623_1'
cal_path = f'/home/evanmayer/TIME_data/mirror_mapping/testing/{run}/calibration/20230629_0'

extensions = ['.jpeg', '.jpg', '.JPG', '.PNG']#, '.tiff', '.TIFF']
files = []
for ext in extensions:
    files += sorted(glob.glob(os.path.join(cal_path, '**/**' + ext), recursive=True))
assert files, 'No image files found when searching for images for camera calibration.'

# check for pickled camera matrices to avoid expensive recalibration
if not (
    os.path.exists(os.path.join(cal_path, 'camera_cal_mtx.pickle')) and
    os.path.exists(os.path.join(cal_path, 'camera_cal_dist.pickle')) and
    os.path.exists(os.path.join(cal_path, 'camera_cal_optimal_camera_matrix.pickle'))
):
    # calibrate the camera for distortion
    mtx, dist, optimal_camera_matrix, roi = pg.calibrate_camera(
        cal_path,
        files,
        square_size=pg.BOARD_SQUARE_SIZE,
        aruco_size=pg.BOARD_ARUCO_SIZE,
        guess_intrinsics=True,
        plot=True,
        savefig=False,
        write_mrcal=False,
    )

    with open(os.path.join(cal_path, 'camera_cal_mtx.pickle'), 'wb') as f:
        pickle.dump(mtx, f, protocol=pickle.HIGHEST_PROTOCOL)
    with open(os.path.join(cal_path, 'camera_cal_dist.pickle'), 'wb') as f:
        pickle.dump(dist, f, protocol=pickle.HIGHEST_PROTOCOL)
    with open(os.path.join(cal_path, 'camera_cal_optimal_camera_matrix.pickle'), 'wb') as f:
        pickle.dump(optimal_camera_matrix, f, protocol=pickle.HIGHEST_PROTOCOL)
else:
    with open(os.path.join(cal_path, 'camera_cal_mtx.pickle'), 'rb') as f:
        mtx = pickle.load(f)
    with open(os.path.join(cal_path, 'camera_cal_dist.pickle'), 'rb') as f:
        dist = pickle.load(f)
    with open(os.path.join(cal_path, 'camera_cal_optimal_camera_matrix.pickle'), 'rb') as f:
        optimal_camera_matrix = pickle.load(f)

print('Camera matrix:', mtx)
print('Distortion matrix:', dist)
mtx_boofcv = np.array([
    [1653.5405560649892, 0, 2011.1627804232648],
    [0, 1655.631444381979,  1515.845563652173],
    [0, 0, 1]
])
print('Mine:\n', cv2.calibrationMatrixValues(mtx, (4032, 3024), 1.7e-6 * 4032, 1.7e-6 * 3024))
print('vs. BoofCV:\n', cv2.calibrationMatrixValues(mtx_boofcv, (4032, 3024), 1.7e-6 * 4032, 1.7e-6 * 3024))

print('Mine: (may not share distortion model with BoofCV)\n', dist)
dist_boofcv = np.array([
    -0.001963787455093832,
    0.0026115087601481127,
    -0.015169613897069615,
    0.02001275069414555,
    -0.01121304725629186,
    0.0023117429319429944
])
print('vs. BoofCV:\n', dist_boofcv)

`sort_images_exif ./ ./`

In [ ]:
meas_path = f'/home/evanmayer/TIME_data/mirror_mapping/testing/{run}/measurement/'
files = []
for ext in extensions:
    files += sorted(glob.glob(os.path.join(meas_path, '**' + ext)))
assert files, 'No image files found when searching for images for measurement.'

# If necessary, skip some images (e.g. interleaved visits of the home position)
slice = np.s_[0::2] # scans
# slice = np.s_[1::2] # repeatability
# slice = np.s_[:] # all
files = files[slice]
print(len(files))

In [ ]:
# use camera cal matrix to de-distort a few images to check
for file in files[:2]:
    fig, ax = plt.subplots(figsize=(12,7))
    ax.imshow(pg.load_to_gray(file, mtx, dist), cmap='bone')

In [ ]:
origin_id = 998
raft_id = 999

In [ ]:
%autoreload 2

img_data = pg.measure_images(files, mtx, dist, aruco_ids=[origin_id, raft_id], use_extrinsic_priors=False, plot=False)
# img_data = np.load('/home/evanmayer/TIME_data/mirror_mapping/testing/20230623_1/results_recal/1694456444_9875538_img_data.pickle', allow_pickle=True)

In [ ]:
fig, ax = plt.subplots(figsize=(12,5), subplot_kw={'projection':'3d'})
ax.view_init(elev=-15, azim=145, roll=0)

for file, entities in img_data.items():
    for id, pose in entities.items():
        for key, val in pose.items():
            print(os.path.basename(file), id, key, val)
            if key == 'tvec_rel_camera':
            # if key == 'tvec':
                if id == raft_id:
                    color = 'k'
                elif id == origin_id:
                    color = 'r'
                elif id == 'board':
                    color = 'b'
                else:
                    continue
                ax.scatter(
                    val[0],
                    val[1],
                    val[2],
                    color=color,
                    # label=id,
                    alpha=0.7
                )

ax.scatter(0,0,0, color='m', label='origin')
ax.set(xlabel='x', ylabel='y', zlabel='z')
ax.set_box_aspect([1,1,1])
# ax.legend()
# origin = np.array([-0.27714734, 0.28417057, 0.36009227])
# sz = .01
# ax.set_xlim(-sz + origin[0], sz + origin[0])
# ax.set_ylim(-sz + origin[1], sz + origin[1])
# ax.set_zlim(-sz + origin[2], sz + origin[2]);

board_median_tvec_rel_camera = np.median(pg.unwind_img_data(img_data, 'board', 'tvec_rel_camera'), axis=0)
board_median_rvec_rel_camera = np.median(pg.unwind_img_data(img_data, 'board', 'rvec_rel_camera'), axis=0)
print(board_median_tvec_rel_camera)
ax.scatter(board_median_tvec_rel_camera[0], board_median_tvec_rel_camera[1], board_median_tvec_rel_camera[2], color='limegreen', zorder=0)

In [ ]:
def plot_xyz_scatter(vals):
    median = np.median(vals, axis=0)
    std = np.std(vals, axis=0)
    [print(f'{m:.4E}') for m in median]
    print()
    [print(f'{s:.4E}') for s in std]

    sz = 2e-3

    fig, ax = plt.subplots(ncols=3)
    ax[0].scatter(vals[:,0], vals[:,1], alpha=0.2)
    ax[0].scatter(median[0], median[1], color='r')
    ax[0].set_xlim(median[0]-sz, median[0]+sz)
    ax[0].set_ylim(median[1]-sz, median[1]+sz)
    ax[0].set(xlabel='X', ylabel='Y')

    ax[1].scatter(vals[:,1], vals[:,2], alpha=0.2)
    ax[1].scatter(median[1], median[2], color='r')
    ax[1].set_xlim(median[1]-sz, median[1]+sz)
    ax[1].set_ylim(median[2]-sz, median[2]+sz)
    ax[1].set(xlabel='Y', ylabel='Z')

    ax[2].scatter(vals[:,0], vals[:,2], alpha=0.2)
    ax[2].scatter(median[0], median[2], color='r')
    ax[2].set_xlim(median[0]-sz, median[0]+sz)
    ax[2].set_ylim(median[2]-sz, median[2]+sz)
    ax[2].set(xlabel='X', ylabel='Z')

    for i, a in enumerate(ax):
        a.set_aspect('equal')

    fig.tight_layout()
    return fig, ax

In [ ]:
vals = pg.unwind_img_data(img_data, 'board', 'tvec_rel_camera')
fig, ax = plot_xyz_scatter(vals)

In [ ]:
vals = pg.unwind_img_data(img_data, 998, 'tvec_rel_camera')
fig, ax = plot_xyz_scatter(vals)

In [ ]:
board_eulers = pg.unwind_img_data(img_data, 'board', 'euler_zyx_deg')
euler_zs = board_eulers[:,0]
euler_ys = board_eulers[:,1]
euler_xs = board_eulers[:,2]

fig, ax = plt.subplots(figsize=(12,5), ncols=3)
sns.kdeplot(euler_zs, fill=True, ax=ax[0])
sns.rugplot(euler_zs, ax=ax[0])
ax[0].axvline(np.median(euler_zs), linestyle='--')
ax[0].set_xlabel(f'Euler z Angle (deg)\n{np.median(euler_zs):.2f} +/- {np.std(euler_zs):.2f}', fontsize=14)
ax[0].yaxis.label.set_size(14)

sns.kdeplot(euler_ys, fill=True, ax=ax[1])
sns.rugplot(euler_ys, ax=ax[1])
ax[1].axvline(np.median(euler_ys), linestyle='--')
ax[1].set_xlabel(f'Euler y Angle (deg)\n{np.median(euler_ys):.2f} +/- {np.std(euler_ys):.2f}', fontsize=14)
ax[1].yaxis.label.set_size(14)

sns.kdeplot(euler_xs, fill=True, ax=ax[2])
sns.rugplot(euler_xs, ax=ax[2])
ax[2].axvline(np.median(euler_xs), linestyle='--')
# ax.set_xlim(-181, -179)
ax[2].set_xlabel(f'Euler x Angle (deg)\n{np.median(euler_xs):.2f} +/- {np.std(euler_xs):.2f}', fontsize=14)
ax[2].yaxis.label.set_size(14)

fig.suptitle('ChArUco Board')

timestamp = str(time.time()).replace('.', '_')
# fig.savefig(
#     os.path.join(
#         f'/home/evanmayer/TIME_data/mirror_mapping/testing/{run}/results_recal/',
#         f'{timestamp}_eulers.png'
#     ),
#     facecolor='white',
#     transparent=False,
#     bbox_inches='tight'
# )

In [ ]:
origin_eulers = pg.unwind_img_data(img_data, origin_id, 'euler_zyx_deg')
origin_euler_zs = origin_eulers[:,0]
origin_euler_ys = origin_eulers[:,1]
origin_euler_xs = origin_eulers[:,2]

fig, ax = plt.subplots(figsize=(12,5), ncols=3)
sns.kdeplot(origin_euler_zs, fill=True, ax=ax[0])
sns.rugplot(origin_euler_zs, ax=ax[0])
ax[0].axvline(np.median(origin_euler_zs), linestyle='--')
ax[0].set_xlabel(f'Euler z Angle (deg)\n{np.median(origin_euler_zs):.2f} +/- {np.std(origin_euler_zs):.2f}', fontsize=14)
ax[0].yaxis.label.set_size(14)

sns.kdeplot(origin_euler_ys, fill=True, ax=ax[1])
sns.rugplot(origin_euler_ys, ax=ax[1])
ax[1].axvline(np.median(origin_euler_ys), linestyle='--')
ax[1].set_xlabel(f'Euler y Angle (deg)\n{np.median(origin_euler_ys):.2f} +/- {np.std(origin_euler_ys):.2f}', fontsize=14)
ax[1].yaxis.label.set_size(14)

sns.kdeplot(origin_euler_xs, fill=True, ax=ax[2])
sns.rugplot(origin_euler_xs, ax=ax[2])
ax[2].axvline(np.median(origin_euler_xs), linestyle='--')
# ax.set_xlim(-181, -179)
ax[2].set_xlabel(f'Euler x Angle (deg)\n{np.median(origin_euler_xs):.2f} +/- {np.std(origin_euler_xs):.2f}', fontsize=14)
ax[2].yaxis.label.set_size(14)

fig.suptitle('Origin')

timestamp = str(time.time()).replace('.', '_')
# fig.savefig(
#     os.path.join(
#         f'/home/evanmayer/TIME_data/mirror_mapping/testing/{run}/results_recal/',
#         f'{timestamp}_origin_eulers.png'
#     ),
#     facecolor='white',
#     transparent=False,
#     bbox_inches='tight'
# )

Use this cell if processing scan data. Otherwise, comment out.

In [ ]:
%autoreload
import importlib

plt.style.use('seaborn-v0_8-paper')
importlib.reload(pg)

timestamp = str(time.time()).replace('.', '_')

xs, ys, commanded_pts, measured_pts, camera_dist = pg.post_process_scan(
    img_data,
    f'/home/evanmayer/TIME_data/mirror_mapping/testing/{run}/results_recal/',
    os.path.join('..', 'data', 'input', 'profiles', '25in_breadboard_raster_skipline_10x10_0.40mx0.40m.csv'),
    raft_id,
    origin_id=origin_id,
    timestamp=timestamp
)
pg.make_plots(
    f'/home/evanmayer/TIME_data/mirror_mapping/testing/{run}/results_recal/',
    xs,
    ys,
    commanded_pts,
    measured_pts,
    camera_dist,
    filter_results=True,
    timestamp=timestamp,
    savefig=False
)

Use this cell if processing repeatability data. Otherwise, comment out.

In [ ]:
# %autoreload

# timestamp = str(time.time()).replace('.', '_')

# commanded_pts, measured_pts = pg.post_process_repeatability(
#     img_data,
#     f'/home/evanmayer/TIME_data/mirror_mapping/testing/{run}/results_recal/',
#     os.path.join('..', 'data', 'input', 'profiles', '25in_breadboard_raster_skipline_many_tie_point_10x10_0.40mx0.40m.csv'),
#     raft_id,
#     origin_id=origin_id,
#     timestamp=timestamp,
#     savefig=False
# )
# pg.make_repeatability_plots(
#     f'/home/evanmayer/TIME_data/mirror_mapping/testing/{run}/results_recal/',
#     commanded_pts,
#     measured_pts,
#     timestamp=timestamp,
#     savefig=False
# )

Uncomment this cell if you'd like to observe the stability of the a particular entity in the given dataset.

In [ ]:
# %autoreload

# timestamp = str(time.time()).replace('.', '_')

# measured_pts = pg.post_process_measurement_error(
#     img_data,
#     f'/home/evanmayer/TIME_data/mirror_mapping/testing/{run}/results_recal/',
#     origin_id,
#     timestamp=timestamp,
#     savefig=False
# )
# pg.make_measurement_error_plots(
#     f'/home/evanmayer/TIME_data/mirror_mapping/testing/{run}/results_recal/',
#     measured_pts,
#     origin_id,
#     timestamp=timestamp,
#     savefig=False
# )

In [ ]:
# %autoreload

# timestamp = str(time.time()).replace('.', '_')

# measured_pts = pg.post_process_measurement_error(
#     img_data,
#     f'/home/evanmayer/TIME_data/mirror_mapping/testing/{run}/results_recal/',
#     'board',
#     timestamp=timestamp,
#     savefig=False
# )
# pg.make_measurement_error_plots(
#     f'/home/evanmayer/TIME_data/mirror_mapping/testing/{run}/results_recal/',
#     measured_pts,
#     'board',
#     timestamp=timestamp,
#     savefig=False
# )